### Overview
This notebook demonstrates the basic implementation of **Google Generative AI LLM** in **LangChain**, showcasing how to use embeddings for a **Medical Chatbot** project.

### Project Details
The full project is available on GitHub:  
[Medical Chatbot for FAQ Repository](https://github.com/Chaimaaorg/Medical-Chatbot-for-FAQ.git)

**Libraries**

In [1]:
!pip install transformers==4.48.2
!pip install sentence-transformers==3.4.1
!pip install huggingface_hub==0.28.1
!pip install google-ai-generativelanguage==0.6.15 
!pip install google-generativeai==0.8.4
!pip install faiss-gpu==1.7.2
!pip install langchain==0.3.17
!pip install langchain-google-genai==2.0.9
!pip install langchain-community==0.3.16

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4.47.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.9/275.9 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.1/464.1 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.27.0
    Uninstalling huggingface-hub-0.27.0:
      Successfully uninstalled huggingface-hub-0.27.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.6 MB/s eta 0:00:00
  Attempting uninstall: 

In [2]:
from kaggle_secrets import UserSecretsClient
from langchain_google_genai import ChatGoogleGenerativeAI

# Récupérer la clé API depuis Kaggle Secrets
user_secrets = UserSecretsClient()
api_key = user_secrets.get_secret("GOOGLE_API_KEY")

# Initialisation du modèle Gemini
llm = ChatGoogleGenerativeAI(
    google_api_key=api_key,
    temperature=0.1,
    model="models/gemini-1.5-flash"
)


In [3]:
poem = llm.invoke("Write a 4 line poem of my love for samosa")
print(poem.content)

Golden brown, a crispy shell,
Spiced potato, magic spell.
Savory delight, a perfect bite,
My love for samosa, shining bright.


In [4]:
essay = llm.invoke("write email requesting refund for electronic item")
print(essay.content)

Subject: Refund Request - Order # [Your Order Number] - [Item Name]

Dear [Company Name] Customer Service,

I am writing to request a refund for an electronic item I purchased on [Date of Purchase] with order number [Your Order Number]. The item in question is a [Item Name] ([Item Model Number, if applicable]).

[Choose ONE of the following options, and tailor it to your situation.  Be specific and provide details!]

**Option 1: Faulty Item**

The [Item Name] arrived [Date of Arrival] and is faulty.  [Describe the fault clearly and concisely.  E.g., "The power button does not work," or "The screen flickers constantly," or "It arrived damaged."]. I have attached [photos/videos] as evidence.

**Option 2: Item Not as Described**

The [Item Name] I received does not match the description on your website. [Describe the discrepancy. E.g., "The advertised specifications stated it had 16GB of RAM, but it only has 8GB," or "The color is significantly different from the image shown online."].

*

In [5]:
from langchain.chains import RetrievalQA


from langchain.embeddings import GooglePalmEmbeddings
from langchain.llms import GooglePalm

### Now let's load data from Codebasics FAQ csv file

In [6]:
import pandas as pd
df = pd.read_csv('/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv')
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,6361820,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."


In [7]:
df = df.drop(['Question_ID'],axis=1)

In [8]:
df

,Questions,Answers
0,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,Who does mental illness affect?,It is estimated that mental illness affects 1 ...
2,What causes mental illness?,It is estimated that mental illness affects 1 ...
3,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
4,Can people with mental illness recover?,"When healing from mental illness, early identi..."
...,...,...
93,How do I know if I'm drinking too much?,Sorting out if you are drinking too much can b...
94,"If cannabis is dangerous, why are we legalizin...","Cannabis smoke, for example, contains cancer-c..."
95,How can I convince my kids not to use drugs?,You can't. But you can influence their capacit...
96,What is the legal status (and evidence) of CBD...,Cannabidiol or CBD is a naturally occurring co...


In [9]:
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/kaggle/input/mental-health-faq-for-chatbot/Mental_Health_FAQ.csv', source_column="Questions")

# Store the loaded data in the 'data' variable
data = loader.load()

### Hugging Face Embeddings

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings

instructor_embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")
e = instructor_embeddings.embed_query("What is your refund policy?")

<ipython-input-10-427c2ce4a05b>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  instructor_embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/paraphrase-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
len(e)

384

In [12]:
e[:5]

[-0.43751096725463867,
 -0.5132736563682556,
 0.2075427621603012,
 -0.048874497413635254,
 0.8277162909507751]

### Vector store using FAISS

In [13]:
from langchain.vectorstores import FAISS

# Create a FAISS instance for vector database from 'data'
vectordb = FAISS.from_documents(documents=data,
                                 embedding=instructor_embeddings)

# Create a retriever for querying the vector database
retriever = vectordb.as_retriever(score_threshold = 0.7)

In [14]:
rdocs = retriever.get_relevant_documents("How would you describe what it's like to live with a mental illness?")
rdocs

<ipython-input-14-70cd83e74b6f>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  rdocs = retriever.get_relevant_documents("How would you describe what it's like to live with a mental illness?")


[Document(id='14aef51b-b4e0-4bdd-a07e-3d2379909308', metadata={'source': 'What does it mean to have a mental illness?', 'row': 0}, page_content='Question_ID: 1590140\nQuestions: What does it mean to have a mental illness?\nAnswers: Mental illnesses are health conditions that disrupt a personâ€™s thoughts, emotions, relationships, and daily functioning. They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.\nMental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life. These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.\nIt is important to know that mental illnesses are medical conditions that have nothing to do with a

As you can see above, the retriever that was created using FAISS and hugging face embedding is now capable of pulling relavant documents from our original CSV file knowledge store. This is very powerful and it will help us further in our project

### Create RetrievalQA chain along with prompt template 🚀

In [15]:
from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


### We are all set 👍🏼 Let's ask some questions now

In [16]:
chain("Could you describe the impact of mental illness on a person's life?")

<ipython-input-16-1adad9cea183>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain("Could you describe the impact of mental illness on a person's life?")


{'query': "Could you describe the impact of mental illness on a person's life?",
 'result': 'Mental illnesses are health conditions that disrupt a person’s thoughts, emotions, relationships, and daily functioning.  They are associated with distress and diminished capacity to engage in the ordinary activities of daily life.  Mental illnesses fall along a continuum of severity: some are fairly mild and only interfere with some aspects of life, such as certain phobias. On the other end of the spectrum lie serious mental illnesses, which result in major functional impairment and interference with daily life.  These include such disorders as major depression, schizophrenia, and bipolar disorder, and may require that the person receives care in a hospital.',
 'source_documents': [Document(id='14aef51b-b4e0-4bdd-a07e-3d2379909308', metadata={'source': 'What does it mean to have a mental illness?', 'row': 0}, page_content='Question_ID: 1590140\nQuestions: What does it mean to have a mental ill

**As you can see above, the answer of question comes from two different FAQs within our csv file and it is able to pull those questions and merge them nicely**

In [17]:
chain("What is the scientific consensus on the effects of vaping?")

{'query': 'What is the scientific consensus on the effects of vaping?',
 'result': 'Vaping is less harmful than smoking cigarettes and evidence suggests it may help some people quit smoking.  Evidence suggests vaping is likely not a gateway to smoking tobacco. Though less harmful than smoking cigarettes, vaping still carries potential harms. There is a small exposure to nicotine and possible carcinogens in the vapor. Some evidence suggests that young people may be unaware the vaping liquid often contains nicotine. E-cigarettes are not controlled and the nicotine content of vaping liquids varies significantly among brands. Other chemicals, flavourings, and heavy metals present in vaping liquids, have not as yet been well studied. Long-term health risks from vaping are thus undetermined. More research is needed in these areas.',
 'source_documents': [Document(id='ddab1bf0-9f51-4220-8c75-06c403341d10', metadata={'source': 'What is the evidence on vaping?', 'row': 97}, page_content='Questi